In [63]:
from scipy.spatial import distance as dist
from collections import OrderedDict
import numpy as np

# from typing import List
# from numpy.typing import NDArray

In [64]:
class CentroidTracker():
    def __init__(self, maxDisappeared=50):
        self.nextObjectId=0                 # Id is number of object it holds. ex) self.nextObjectId=6 means 5 objects are cared by this class, next new object's id is 6.

        self.objects = OrderedDict()        # save data as self.obejcts[Id] = centroid. centroid is center of object, type is (x : int, y : int)
        self.disappeared = OrderedDict()    # self.disappeared[Id] = counts. if this reached self.maxDisappeared, Judge this object as disappeared and delete it.

        self.maxDisappeared = maxDisappeared

    def register(self, centroid):
        self.objects[self.nextObjectId] = centroid      # add new centroid and id is current self.nextObjectId
        self.disappeared[self.nextObjectId] = 0         # set self.nextObjectId 
        self.nextObjectId += 1                          # 


    def deregister(self, objectId):
        del self.objects[objectId]
        del self.disappeared[objectId]


    # format of rects = np.array of (startX, startY, endX, endY)
    # update is called every frame. it receive rects and update disappeared and objects
    # update(self, )
    def update(self, rects):  
        # there's no detection. add all object's disappeared by 1                                         
        if len(rects) == 0:
            for objectId in list(self.disappeared.keys()):
                self.disappeared[objectId] += 1
            
                if self.disappeared[objectId] > self.maxDisappeared:
                    self.deregister(objectId)

            return self.objects
        
        # make rooms for input_centroid.
        input_centroids = np.zeros((len(rects),2), dtype="int")

        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            cX = int((startX+endX) / 2.0)
            cY = int((startY+endY) / 2.0)
            input_centroids[i] = (cX, cY)

        # if objects are empty
        if len(self.objects) == 0:
            for centroid in input_centroids:
                self.register(centroid)
            
        else:
            objectIds = list(self.objects.keys())
            object_centroids = list(self.objects.values())

            D = dist.cdist(np.array(object_centroids), input_centroids)

            rows = D.min(axis=1).argsort()

            cols = D.argmin(axis=1)[rows]

            usedRows = set()
            usedCols = set()

            for (row, col) in zip(rows, cols):
                if row in usedRows or col in usedCols:
                    continue

                objectId = objectIds[row]
                self.objects[objectId] = input_centroids[col]
                self.disappeared[objectId] = 0

                usedRows.add(row)
                usedCols.add(col)
            
            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)
            
            if D.shape[0] >= D.shape[1]:
                for row in unusedRows:
                    objectID = objectIds[row]
                    self.disappeared[objectID] += 1

                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)
            else :
                for col in unusedCols:
                    self.register(input_centroids[col])

        return self.objects

                    


In [65]:
from imutils.video import VideoStream
import cv2
import time
import imutils


In [66]:
ct = CentroidTracker()
(H, W) = (None, None)

print("[INFO] : loading model")
net = cv2.dnn.readNetFromCaffe("./deploy.prototxt", "./res10_300x300_ssd_iter_140000.caffemodel")

print("[INFO] : starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)

[INFO] : loading model
[INFO] : starting video stream...


In [77]:
# while True:
# read the next frame from the video stream and resize it
frame = vs.read()
frame = imutils.resize(frame, width=400)
# if the frame dimensions are None, grab them
if W is None or H is None:
    (H, W) = frame.shape[:2]
# construct a blob from the frame, pass it through the network,
# obtain our output predictions, and initialize the list of
# bounding box rectangles
blob = cv2.dnn.blobFromImage(frame, 1.0, (W, H),
    (104.0, 177.0, 123.0))
net.setInput(blob)
detections = net.forward()
rects = []

In [73]:
detections.shape

(1, 1, 200, 7)

In [72]:
import matplotlib.pyplot as plt
from IPython.display import display, Image

In [ ]:
for i in range(0, detections.shape[2]):
	# filter out weak detections by ensuring the predicted
	# probability is greater than a minimum threshold
	if detections[0, 0, i, 2] > 0.5:
		# compute the (x, y)-coordinates of the bounding box for
		# the object, then update the bounding box rectangles list
		box = detections[0, 0, i, 3:7] * np.array([W, H, W, H])
		rects.append(box.astype("int"))
		# draw a bounding box surrounding the object so we can
		# visualize it
		(startX, startY, endX, endY) = box.astype("int")
		cv2.rectangle(frame, (startX, startY), (endX, endY),
			(0, 255, 0), 2)

plt.imshow(cv2.cvtColor(frame,cv2.COLOR_RGB2BGR))